## Semantic Similarity

This notebook walks through the steps to compute word embeddings for each comment in the Reddit dataset. Continuing from the previous notebook, we consider weekly-level data from the months of May and June across the selected subreddits to compute the word embeddings. We utilize an sentence-BERT model using which we compute embeddings and then generate a Semantic Similarity Measure.

----

Install libraries

In [1]:
pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 40.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 84.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
#Download the SBERT model to deploy on the 'body' and the 'receiver_body' columns, so that the word embeddings can be obtained.
!pip install -U sentence-transformers
# load tqdm
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Using cached https://github.com/chengs/tqdm/archive/colab.zip
  Preparing metadata (setup.py) ... - done
  Created wheel for tqdm: filename=tqdm-4.28.1-py2.py3-none-any.whl size=47874 sha256=b5a92152648fdec95a70d620d820513b99cc00434984794bf17ca78c9a317dcf
  Stored in directory: /tmp/pip-ephem-wheel-cache-eycluf5i/wheels/a7/30/17/f85112bf3ff9d07da5ef0a111c02e4a86df55d990ce52864cd
Successfully built tqdm
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.4
    Uninstalling tqdm-4.66.4:
      Successfully uninstalled tqdm-4.66.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [5]:
import numpy as np
import pandas as pd
import nltk
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses, util
from torch.utils.data import DataLoader
from sentence_transformers import losses
import os
import swifter
from nltk.tokenize import sent_tokenize
import torch
nltk.download('punkt')
from tqdm import tqdm
tqdm.pandas()
import pickle

[nltk_data] Downloading package punkt to /home/datalore/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('all-mpnet-base-v2')

We then define functions to generate the word embeddings at the comment level for each week of data. 

In [ ]:
def generate_language_similarity(val1, val2):
    return util.dot_score(val1, val2)

#define function to generate embeddings for one row
def generate_embeddings(record):
   return sbert_model.encode(record, convert_to_numpy=True, normalize_embeddings=True)

def divide_data_3_parts(week_data):
    """
    Divide the data of a given week into three parts
    Parameters
    ----------
    week_data : dataframe
        the data for a given week
    Returns
    ----------
    part1, part2, part3: tuple of dataframes
         the data for each part
    """
    week_data.head(3)
    div, mod = divmod(len(week_data), 3)
    part1 = week_data[:div]
    part2 = week_data[div:div * 2]
    part3 = week_data[div * 2:]
    if mod == 1:
        part3 = pd.concat([part3, part2.iloc[-1:]], ignore_index=True)
        part2 = part2.iloc[:-1]
    elif mod == 2:
        part3 = pd.concat([part3, part2.iloc[-2:]], ignore_index=True)
        part2 = part2.iloc[:-2]
    return part1, part2, part3

def word_embeddings(week_no,part_data,part_no):
    """
    Compute the word embeddings for each author's and receiver's comment and then compute the semantic similarity
    Parameters
    ----------
    week_no : int
        the week's number (from 1 to 26)
    part_data : dataframe
        the subset of the reddit data for the given week
    part_no : int
        the part number (1, 2, or 3)
    Returns
    ----------
        None
    """
    #call the function to generate word embeddings
    print('start generating the word embeddings')
    body_embeddings = part_data['body'].progress_apply(generate_embeddings)
    print('finished generation of body embeddings')
    print(len(body_embeddings))
    receiver_body_embeddings = part_data['receiver_body'].progress_apply(generate_embeddings)
    print('finished generation of receiver embeddings')
    print(len(receiver_body_embeddings))
    print('finished generation of word embeddings')
    model_storage_name_location = 'word_embeddings/embeddings_w'+ str(week_no) +'_part'+str(part_no)+ '.pkl'
    # Open the file in binary write mode
    with open(model_storage_name_location, 'wb') as f:
        pickle.dump(body_embeddings, f)
    print(type(body_embeddings))
    model_storage_name_location_receiver = 'word_embeddings/receiver_embeddings_w'+ str(week_no) +'_part'+str(part_no)+ '.pkl'
    # Open the file in binary write mode
    with open(model_storage_name_location_receiver, 'wb') as f:
        pickle.dump(receiver_body_embeddings, f)
    print(type(receiver_body_embeddings))
    #compute the languageSimilarity
    df1 = pd.concat([body_embeddings, receiver_body_embeddings], axis=1)
    print(len(df1))
    df1['languageSimilarity_commentLevel'] = df1.progress_apply(lambda x: (util.dot_score(x[0],x[1])).item(), axis=1)
    extracted_col = df1['languageSimilarity_commentLevel']
    print(len(extracted_col))
    part_data_final = pd.concat([part_data, extracted_col], axis=1)
    print(len(part_data_final))
    #part_data_final.head(3)
    resulting_file_name = 'bothSim_Processed/mayjune_w'+str(week_no)+'_part'+str(part_no)+ '_processed.parquet'
    part_data_final.to_parquet(resulting_file_name)
    print('completed!')

In [7]:
##############################
#read the embeddings
# week = 1
# part = 3
# body_embeddings_1 = pd.read_pickle('JanFebSubreddits2022/word_embeddings/embeddings_w'+str(week)+'_part'+str(part)+'.pkl')
# print(len(body_embeddings_1))
# print(body_embeddings_1[0])
# print(type(body_embeddings_1))
# receiver_embeddings_1 = pd.read_pickle('JanFebSubreddits2022/word_embeddings/receiver_embeddings_w'+str(week)+'_part'+str(part)+'.pkl')
# print(len(receiver_embeddings_1))
# print(receiver_embeddings_1[0])
# print(type(receiver_embeddings_1))

In [8]:
#############################
#read the processed file with both network similarity and language similarity
# week = 1
# part = 3
# file_path1 = 'JanFebSubreddits2022/bothSim_Processed/janfeb_w' + str(week) + '_part'+str(part)+'_processed.parquet'
# week_data2 = pd.read_parquet(file_path1)
# print(week_data2['subreddit'].value_counts())
# print(week_data2['date'].value_counts())
# week_data2.head(2)

To demonstrate, we show how we call the functions to generate word embeddings and hence a semantic similarity measure using the above functions, for a subset of data.

## Process Week 18.

In [8]:
#reading the data and dividing it into parts
w18 = pd.read_parquet('netSim_processed/mayjune_w18_processed.parquet')
print(len(w18))
w18_p1,w18_p2,w18_p3 = divide_data_3_parts(w18)
print(len(w18_p1))
print(len(w18_p2))
print(len(w18_p3))
print(len(w18_p1)+len(w18_p2)+len(w18_p3))
w18['subreddit'].value_counts()

1008518
336172
336170
336176
1008518


amitheasshole 372661
politics 257921
worldnews 230104
news 118196
science 29636
Name: subreddit, dtype: int64

In [9]:
w18['date'].value_counts()

2022-05-03 222768
2022-05-04 162014
2022-05-02 144216
2022-05-05 140083
2022-05-06 139102
2022-05-07 117116
2022-05-01 83219
Name: date, dtype: int64

In [3]:
#w18.head(2)

In [14]:
word_embeddings(18,w18_p1,1)

start generating the word embeddings
finished generation of body embeddings
336172
finished generation of receiver embeddings
336172
finished generation of word embeddings
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
336172
336172
336172
completed!


100%|██████████| 336172/336172 [00:17<00:00, 19733.71it/s]


In [4]:
#read the processed file with both network similarity and language similarity
#week = 18
#part = 1
#file_path1 = 'bothSim_Processed/mayjune_w' + str(week) + '_part'+str(part)+'_processed.parquet'
#test = pd.read_parquet(file_path1)
#test.head(2)